# Automatic Data Access Management 
   In this tutorial we'll explore how to use admin role to set and deploy automatic rules to accept / deny data access requests from remote users.

### 1 - Import libraries

In [ ]:
from syft.grid.grid_client import connect
from syft.grid.connections.http_connection import HTTPConnection
from syft.core.node.domain.client import DomainClient

import syft as sy
import torch as th

sy.VERBOSE = False

### 2 - Connect/Authenticate with the Domain server (Using an admin account)
We need to sign up as a Domain Admin to be able to deploy new automatic request handlers 

In [ ]:
client = connect(
    url="http://localhost:5000", # Domain Address
    credentials= {"email": "admin@email.com", "password": "pwd123"}, # Admin role
    conn_type= HTTPConnection, # HTTP Connection Protocol
    client_type=DomainClient) # Domain Client type

### 3 - Let's create a new dataset sample
To show how it work, we'll create a random dataset sample and host it into the domain server

In [ ]:
age_data = th.rand((9216, 96)) # Create a new / random dataset sample

# Let's tag it as "age data"
age_data = age_data.tag("#age_data")

# Let's add a description to the tensor
age_data = age_data.describe("This is a list of people's ages. Let's keep it private!")

# Let's UPLOAD THE DATA to the Duet server!
age_data_pointer = age_data.send(client, searchable=True) # Search = True makes the pointer "visible" to everyone
print("Dataset Pointer Result: ", age_data_pointer)

### 4 -  Check the automatic request handlers list
As we can see here, our auto request handler queue is empty

In [ ]:
client.requests.handlers

### 5 - Add a new handler allowing data access
Let's create a new handler rule to allow age-data access automatically

In [ ]:
accept_handler = {
    "request_name": "age_data",
    "timeout_secs": -1,
    "action": "accept", # Accept / Deny
    "print_local": True, # Print it in Domain Server
    "log_local": True # Log it in Domain Server
}
client.requests.add_handler(accept_handler)

### 6 -  Check the automatic request handlers list again
As we can see here, our auto request handler queue is not empty anymore

In [ ]:
client.requests.handlers

### ___PS: We're still able to allow/deny data access requests manually, this feature just aims to show how to manage data access automatically (in a context with multiple data access requests, do it using manual approach wouldn't be feasible)___